In [ ]:
# | default_exp _helpers.quarto_to_mkdocs

In [ ]:
# | export

import re
from pathlib import Path
from typing import *
import textwrap

import nbformat

In [ ]:
from tempfile import TemporaryDirectory
import shutil

In [ ]:
# | export


def _update_conditional_content_tags(text: str) -> str:
    """Update conditional content tags.

    Args:
        text: The text to update the conditional content tags in.

    Returns:
        The updated text with the conditional content tags modified.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    pattern = r":::\s*{(?:\s*.*\.content-visible|\s*\.content-hidden)\s*(when|unless)-format\s*=\\\s*(\"|\')\s*(html|markdown)\s*\\(\"|\')\s*.*}"
    text = re.sub(
        pattern,
        lambda m: m.group(0).replace(
            m.group(1), "when" if m.group(1) == "unless" else "unless"
        ),
        text,
    )
    return text

In [ ]:
_input = """
may have some text before ::: {markdown=1 .content-visible when-format=\\"markdown\\" style=\\"text-align: center\\"}

some random text 

::: {.content-visible when-format=\\"markdown\\"}
"""
expected = """
may have some text before ::: {markdown=1 .content-visible unless-format=\\"markdown\\" style=\\"text-align: center\\"}

some random text 

::: {.content-visible unless-format=\\"markdown\\"}
"""

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = ':::  { .content-visible when-format=\\"html\\"}'
expected = ':::  { .content-visible unless-format=\\"html\\"}'

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = ':::  { .content-visible  unless-format=\\"html\\"}'
expected = ':::  { .content-visible  when-format=\\"html\\"}'

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = ":::  { .content-visible  unless-format=\\'markdown\\'}"
expected = ":::  { .content-visible  when-format=\\'markdown\\'}"

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual


may have some text before ::: {markdown=1 .content-visible unless-format=\"markdown\" style=\"text-align: center\"}

some random text 

::: {.content-visible unless-format=\"markdown\"}

:::  { .content-visible unless-format=\"html\"}
:::  { .content-visible  when-format=\"html\"}
:::  { .content-visible  when-format=\'markdown\'}


In [ ]:
_input = ":::  { .content-hidden  when-format=\\'html\\'}"
expected = ":::  { .content-hidden  unless-format=\\'html\\'}"

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = '::: { .content-hidden  when-format=\\"markdown\\" }'
expected = '::: { .content-hidden  unless-format=\\"markdown\\" }'

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = '::: { .content-hidden  unless-format=\\"html\\" }'
expected = '::: { .content-hidden  when-format=\\"html\\" }'

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = '::: { .content-hidden  unless-format=\\"markdown\\" }'
expected = '::: { .content-hidden  when-format=\\"markdown\\" }'

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

:::  { .content-hidden  unless-format=\'html\'}
::: { .content-hidden  unless-format=\"markdown\" }
::: { .content-hidden  when-format=\"html\" }
::: { .content-hidden  when-format=\"markdown\" }


In [ ]:
# | export


def _update_mermaid_chart_tags(text: str) -> str:
    """Convert the mermaid chart tags from quarto format to markdown format.

    Args:
        text: The text to update the mermaid chart tags in.

    Returns:
        The updated text with the mermaid chart tags modified.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    pattern = r"```\s*{mermaid\s*}"
    text = re.sub(pattern, "``` mermaid", text)
    return text

In [ ]:
valid_inputs = [
    "\n```{mermaid}\nflowchart LR\n ",
    "\n\n```   {mermaid}   \nflowchart LR\n ",
]
expected = ["\n``` mermaid\nflowchart LR\n ", "\n\n``` mermaid   \nflowchart LR\n "]

for n, i in enumerate(valid_inputs):
    actual = _update_mermaid_chart_tags(i)
    print(actual)
    assert actual == expected[n], actual


invalid_inputs = [
    "\n```{ mermaid}\nflowchart LR\n ",
    "\n\n```  some text {mermaid}   \nflowchart LR\n ",
]
expected = [
    "\n```{ mermaid}\nflowchart LR\n ",
    "\n\n```  some text {mermaid}   \nflowchart LR\n ",
]

for n, i in enumerate(invalid_inputs):
    actual = _update_mermaid_chart_tags(i)
    print(actual)
    assert actual == expected[n], actual


``` mermaid
flowchart LR
 


``` mermaid   
flowchart LR
 

```{ mermaid}
flowchart LR
 


```  some text {mermaid}   
flowchart LR
 


In [ ]:
# | export


def _add_markdown_attribute_to_enable_md_in_html(text: str) -> str:
    """Add markdown attribute to enable markdown in html.

    Args:
        text: The text to add the markdown attribute to

    Returns:
        The text with the markdown attribute added

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    pattern = r":::\s*{\s*(markdown=1)?\s*"
    text = re.sub(pattern, r"::: {markdown=1 ", text)
    return text

In [ ]:
_input = """
Sample markdown Content :::  {.content-visible when-format="html"}

This is a *Markdown* Paragraph.

:::
Sample markdown Content ::: {markdown=1 .content-visible when-format="html"}
This is a *Markdown* Paragraph.

:::
"""
expected = """
Sample markdown Content ::: {markdown=1 .content-visible when-format="html"}

This is a *Markdown* Paragraph.

:::
Sample markdown Content ::: {markdown=1 .content-visible when-format="html"}
This is a *Markdown* Paragraph.

:::
"""
actual = _add_markdown_attribute_to_enable_md_in_html(_input)
print(actual)

assert actual == expected


Sample markdown Content ::: {markdown=1 .content-visible when-format="html"}

This is a *Markdown* Paragraph.

:::
Sample markdown Content ::: {markdown=1 .content-visible when-format="html"}
This is a *Markdown* Paragraph.

:::



In [ ]:
# | export


def _get_callout_identifier(text: str) -> str:
    """Returns the callout identifier based on the input text.

    Args:
        text (str): The text to parse.

    Returns:
        str: The callout identifier ('???' if 'collapse="true"', '???+' if
            'collapse="false"', '!!!' otherwise).
    """
    if 'collapse="true"' in text or "collapse='true'" in text:
        ret_val = "???"
    elif 'collapse="false"' in text or "collapse='false'" in text:
        ret_val = "???+"
    else:
        ret_val = "!!!"

    return ret_val

In [ ]:
fixtures = [
    "",
    ' collapse="false" title="Some title"',
    ' collapse="true" icon=false appearance="minimal"',
    " collapse='false' title='Some title'",
    " collapse='true' title='Some title'",
]
expected = ["!!!", "???+", "???", "???+", "???"]

for i, fixture in enumerate(fixtures):
    actual = _get_callout_identifier(fixture)
    print(actual)
    assert actual == expected[i]

!!!
???+
???
???+
???


In [ ]:
# | export


def _get_callout_title_and_content(
    callout_attributes: str, callout_content: str
) -> Tuple[Optional[str], str]:
    """Get the title and content of a callout based on its attributes and content.

    Args:
        callout_attributes: The attributes of the callout.
        callout_content: The content of the callout.

    Returns:
        A tuple containing the title of the callout (if any) and the content of the callout.
    """
    title = None
    title_regex = r"title=['\"]([\w\s]+)['\"]"
    if "title=" in callout_attributes:
        match = re.search(title_regex, callout_attributes)
        if match:
            title = match.group(1)
    elif (
        "icon=false" in callout_attributes
        or 'appearance="minimal"' in callout_attributes
        or "appearance='minimal'" in callout_attributes
    ):
        title = ""
    elif callout_content.split("\n")[0].startswith("##"):
        title = callout_content.split("\n")[0].replace("##", "").strip()
        callout_content = "\n".join(callout_content.split("\n")[2:])

    return title, callout_content

In [ ]:
callout_attributes = ".callout-note icon=false"
callout_content = """Using callouts is an effective way to highlight content that your reader give special consideration or attention."""

expected_title = ""
expected_callout_content = """Using callouts is an effective way to highlight content that your reader give special consideration or attention."""

actual_title, actual_callout_content = _get_callout_title_and_content(callout_attributes, callout_content)
print(f"{actual_title}")
print(f"{actual_callout_content}")
assert actual_title == expected_title
assert actual_callout_content == expected_callout_content

callout_attributes = '.callout-note appearance="minimal"'
callout_content = """Using callouts is an effective way to highlight content that your reader give special consideration or attention."""

expected_title = ""
expected_callout_content = """Using callouts is an effective way to highlight content that your reader give special consideration or attention."""

actual_title, actual_callout_content = _get_callout_title_and_content(callout_attributes, callout_content)
print(f"{actual_title}")
print(f"{actual_callout_content}")
assert actual_title == expected_title
assert actual_callout_content == expected_callout_content


Using callouts is an effective way to highlight content that your reader give special consideration or attention.

Using callouts is an effective way to highlight content that your reader give special consideration or attention.


In [ ]:
callout_attributes = " title='Tip with Title' icon='false'"
callout_content = """## Tip with Title

    Note that there are five types of callouts, including:
    `note`, `warning`, `important`, `tip`, and `caution`.
"""

expected_title = "Tip with Title"
expected_callout_content = """## Tip with Title

    Note that there are five types of callouts, including:
    `note`, `warning`, `important`, `tip`, and `caution`.
"""

actual_title, actual_callout_content = _get_callout_title_and_content(callout_attributes, callout_content)
print(f"{actual_title}")
print(f"{actual_callout_content}")
assert actual_title == expected_title
assert actual_callout_content == expected_callout_content

Tip with Title
## Tip with Title

    Note that there are five types of callouts, including:
    `note`, `warning`, `important`, `tip`, and `caution`.



In [ ]:
callout_attributes = " icon='false'"
callout_content = """## Tip with Title

This is an example of a callout with a title.
"""

expected_title = "Tip with Title"
expected_callout_content = """This is an example of a callout with a title.
"""
actual_title, actual_callout_content = _get_callout_title_and_content(callout_attributes, callout_content)
print(f"{actual_title}")
print(f"{actual_callout_content}")
assert actual_title == expected_title
assert actual_callout_content == expected_callout_content


Tip with Title
This is an example of a callout with a title.



In [ ]:
callout_attributes = ""
callout_content = """

    This is an example of a callout with a title.
"""

expected_title = None
expected_callout_content = """

    This is an example of a callout with a title.
"""
actual_title, actual_callout_content = _get_callout_title_and_content(callout_attributes, callout_content)
print(f"{actual_title}")
print(f"{actual_callout_content}")
assert actual_title == expected_title
assert actual_callout_content == expected_callout_content


None


    This is an example of a callout with a title.



In [ ]:
# | export

MKDOCS_CALLOUT_TEMPLATE_WITHOUT_TITLE = """{callout_identifier} {callout_type}

{callout_content}"""

MKDOCS_CALLOUT_TEMPLATE_WITH_TITLE = """{callout_identifier} {callout_type} {callout_title}

{callout_content}"""

def _fix_callout_syntax(text: str) -> str:
    """Fix the syntax of callout blocks in the input text

    Args:
        text: The input text for which the callout syntax needs to be changed.

    Returns:
        The text with callout blocks converted to the mkdocs format.
    """
    pattern = r':::\s*\{\.callout-(\w+)([^}]*)\}\s*([\s\S]*?):::'
    matches = re.finditer(pattern, text)
    for match in matches:
        callout_identifier = _get_callout_identifier(match.group(2))
        callout_type = match.group(1)
        callout_content = match.group(3)
        callout_title, callout_content = _get_callout_title_and_content(match.group(2), callout_content)
        
        if callout_title is None:        
            mkdocs_callout = MKDOCS_CALLOUT_TEMPLATE_WITHOUT_TITLE.format(
                callout_identifier=callout_identifier,
                callout_type=callout_type,
                callout_content=textwrap.indent(callout_content, " " * 4),
            )
        else:
            mkdocs_callout = MKDOCS_CALLOUT_TEMPLATE_WITH_TITLE.format(
                callout_identifier=callout_identifier,
                callout_type=callout_type,
                callout_title= f'\\"{callout_title}\\"', 
                callout_content=textwrap.indent(callout_content, " " * 4),
            )

        text = text.replace(match.group(0), mkdocs_callout, 1)
    return text

In [ ]:
fixture = """
:::{.callout-note}
Note that there are five types of callouts, including:
`note`, `warning`, `important`, `tip`, and `caution`.
:::
"""

expected = """
!!! note

    Note that there are five types of callouts, including:
    `note`, `warning`, `important`, `tip`, and `caution`.

"""

actual = _fix_callout_syntax(fixture)
print(actual)

assert actual == expected, actual


!!! note

    Note that there are five types of callouts, including:
    `note`, `warning`, `important`, `tip`, and `caution`.




In [ ]:
fixture = """
::: {.callout-tip}
## Tip with Title

Note that there are five types of callouts, including:
`note`, `warning`, `important`, `tip`, and `caution`.
:::

::: {.callout-tip title="Tip with Title 2"}

This is a callout with a title.
:::

"""

expected = """
!!! tip \\"Tip with Title\\"

    Note that there are five types of callouts, including:
    `note`, `warning`, `important`, `tip`, and `caution`.


!!! tip \\"Tip with Title 2\\"

    This is a callout with a title.


"""

actual = _fix_callout_syntax(fixture)
print(actual)

assert actual == expected, actual


!!! tip \"Tip with Title\"

    Note that there are five types of callouts, including:
    `note`, `warning`, `important`, `tip`, and `caution`.


!!! tip \"Tip with Title 2\"

    This is a callout with a title.





In [ ]:
fixture = """
::: {.callout-caution collapse="true"}
## Expand To Learn About Collapse

This is an example of a 'folded' caution callout that can be expanded by the user. You can use `collapse="true"` to collapse it by default or `collapse="false"` to make a collapsible callout that is expanded by default.
:::

::: {.callout-caution collapse="false"}
## Expand To Learn About Collapse

This is an example of a 'folded' caution callout that can be expanded by the user. You can use `collapse="true"` to collapse it by default or `collapse="false"` to make a collapsible callout that is expanded by default.
:::
"""

expected = """
??? caution \\"Expand To Learn About Collapse\\"

    This is an example of a 'folded' caution callout that can be expanded by the user. You can use `collapse="true"` to collapse it by default or `collapse="false"` to make a collapsible callout that is expanded by default.


???+ caution \\"Expand To Learn About Collapse\\"

    This is an example of a 'folded' caution callout that can be expanded by the user. You can use `collapse="true"` to collapse it by default or `collapse="false"` to make a collapsible callout that is expanded by default.

"""

actual = _fix_callout_syntax(fixture)
print(actual)

assert actual == expected, actual


??? caution \"Expand To Learn About Collapse\"

    This is an example of a 'folded' caution callout that can be expanded by the user. You can use `collapse="true"` to collapse it by default or `collapse="false"` to make a collapsible callout that is expanded by default.


???+ caution \"Expand To Learn About Collapse\"

    This is an example of a 'folded' caution callout that can be expanded by the user. You can use `collapse="true"` to collapse it by default or `collapse="false"` to make a collapsible callout that is expanded by default.




In [ ]:
fixture = """
::: {.callout-note icon=false}

Using callouts is an effective way to highlight content that your reader give special consideration or attention.

:::

::: {.callout-note appearance="minimal"}

Using callouts is an effective way to highlight content that your reader give special consideration or attention.

:::
"""

expected = """
!!! note \\"\\"

    Using callouts is an effective way to highlight content that your reader give special consideration or attention.



!!! note \\"\\"

    Using callouts is an effective way to highlight content that your reader give special consideration or attention.


"""

actual = _fix_callout_syntax(fixture)
print(actual)

assert actual == expected, actual


!!! note \"\"

    Using callouts is an effective way to highlight content that your reader give special consideration or attention.



!!! note \"\"

    Using callouts is an effective way to highlight content that your reader give special consideration or attention.





In [ ]:
# | export

def _fix_callout_syntax_in_file(text: str) -> str:
    """Converts the callout blocks syntax in the given text from quarto format to the mkdocs format.

    Args:
        text: The contents of a QMD file or a notebook in JSON string format.

    Returns:
        The contents of the QMD file or the notebook in JSON string format with the callout blocks syntax converted to the mkdocs format.
    """
    contents: str = ""
    try:
        nb = nbformat.reads(text, as_version=4)
        for cell in nb.cells:
            if cell.cell_type == "markdown":
                original_src = cell["source"]
                modified_src = _fix_callout_syntax(original_src)
                cell["source"] = modified_src

        contents = nbformat.writes(nb)
    except nbformat.reader.NotJSONError as e:
        contents = _fix_callout_syntax(text)
    return contents


In [ ]:
with TemporaryDirectory() as d:

    nbs_path = Path(d) / "nbs"
    nbs_path.mkdir()
    
    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")
    fname = Path(d) / "nbs" / "Test_Notebook_For_Quarto_Syntax_Conversion.qmd"
    shutil.copyfile(
        _root_path / "fixtures" / "Test_Notebook_For_Quarto_Syntax_Conversion.qmd",
        fname,
    )
    
    with open(fname, "r") as f:
        contents = f.read()
    
    actual = _fix_callout_syntax_in_file(contents)
    
    assert ".callout-note" not in actual
    print(actual)

---
title: "Sample"
author: Sample
---

## Introduction

!!! note

    Note that there are five types of callouts, including:
    `note`, `warning`, `important`, `tip`, and `caution`.



In [ ]:
with TemporaryDirectory() as d:

    nbs_path = Path(d) / "nbs"
    nbs_path.mkdir()
    
    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")
    fname = Path(d) / "nbs" / "Test_Notebook_For_Quarto_Syntax_Conversion.ipynb"
    shutil.copyfile(
        _root_path / "fixtures" / "Test_Notebook_For_Quarto_Syntax_Conversion.ipynb",
        fname,
    )
    
    with open(fname, "r") as f:
        contents = f.read()
    
    actual = _fix_callout_syntax_in_file(contents)
    display(actual)
    
    assert ".callout-note" not in actual
    assert ".callout-tip"  not in actual
    assert 'title="Tip with Title"' not in actual
    assert 'collapse="true"' not in actual
    assert 'collapse="false"' not in actual
    assert 'icon=false' not in actual
    assert 'appearance="minimal' not in actual

'{\n "cells": [\n  {\n   "cell_type": "markdown",\n   "id": "8ee6e33a",\n   "metadata": {},\n   "source": [\n    "This notebook is used to test the conversion of Quarto to Material for MkDocs syntax. It serves as a fixture to verify the syntax, including callouts, Mermaid charts, and making a section hidden, which differ between Quarto and Material for MkDocs."\n   ]\n  },\n  {\n   "cell_type": "markdown",\n   "id": "58c93178",\n   "metadata": {},\n   "source": [\n    "# Callouts"\n   ]\n  },\n  {\n   "cell_type": "markdown",\n   "id": "d0a10ffc",\n   "metadata": {},\n   "source": [\n    "!!! note\\n",\n    "\\n",\n    "    Note that there are five types of callouts, including:\\n",\n    "    `note`, `warning`, `important`, `tip`, and `caution`.\\n"\n   ]\n  },\n  {\n   "cell_type": "markdown",\n   "id": "22014eb3",\n   "metadata": {},\n   "source": [\n    "!!! tip \\\\\\"Tip with Title\\\\\\"\\n",\n    "\\n",\n    "    This is an example of a callout with a title.\\n"\n   ]\n  },\n  {

In [ ]:
# | export


def _update_quarto_tags_to_markdown_format(nb_path: Path) -> None:
    """Update Quarto tags to Markdown format

    Args:
        nb_path: Path to the notebook

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    with open(nb_path, "r", encoding="utf-8") as f:
        contents = f.read()

    contents = _update_conditional_content_tags(contents)
    contents = _update_mermaid_chart_tags(contents)
    contents = _fix_callout_syntax_in_file(contents)
    contents = _add_markdown_attribute_to_enable_md_in_html(contents)

    with open(nb_path, "w", encoding="utf-8") as f:
        f.write(contents)

In [ ]:
with TemporaryDirectory() as d:

    nbs_path = Path(d) / "nbs"
    nbs_path.mkdir()
    
    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")
    shutil.copyfile(
        _root_path / "fixtures" / "Test_Notebook_For_Quarto_Syntax_Conversion.ipynb",
        Path(d) / "nbs" / "Test_Notebook_For_Quarto_Syntax_Conversion.ipynb",
    )
    fname = Path(d) / "nbs/Test_Notebook_For_Quarto_Syntax_Conversion.ipynb"
    _update_quarto_tags_to_markdown_format(fname)

    with open(fname, "r") as f:
        contents = f.read()
        
display(contents)

assert '{markdown=1 .content-visible unless-format=\\"markdown\\"}' in contents
assert not '{markdown=1 .content-visible when-format="\\markdown\\"}' in contents

assert "mermaid" in contents
assert not "{mermaid}" in contents

assert '{markdown=1 .content-visible unless-format=\\"html\\"}' in contents
assert not '{markdown=1 .content-visible when-format=\\"html\\"}' in contents

assert ".callout-note" not in contents
assert ".callout-tip"  not in contents
assert 'title="Tip with Title"' not in contents
assert 'collapse="true"' not in contents
assert 'collapse="false"' not in contents
assert 'icon=false' not in contents
assert 'appearance="minimal' not in contents

'{\n "cells": [\n  {\n   "cell_type": "markdown",\n   "id": "8ee6e33a",\n   "metadata": {},\n   "source": [\n    "This notebook is used to test the conversion of Quarto to Material for MkDocs syntax. It serves as a fixture to verify the syntax, including callouts, Mermaid charts, and making a section hidden, which differ between Quarto and Material for MkDocs."\n   ]\n  },\n  {\n   "cell_type": "markdown",\n   "id": "58c93178",\n   "metadata": {},\n   "source": [\n    "# Callouts"\n   ]\n  },\n  {\n   "cell_type": "markdown",\n   "id": "d0a10ffc",\n   "metadata": {},\n   "source": [\n    "!!! note\\n",\n    "\\n",\n    "    Note that there are five types of callouts, including:\\n",\n    "    `note`, `warning`, `important`, `tip`, and `caution`.\\n"\n   ]\n  },\n  {\n   "cell_type": "markdown",\n   "id": "22014eb3",\n   "metadata": {},\n   "source": [\n    "!!! tip \\\\\\"Tip with Title\\\\\\"\\n",\n    "\\n",\n    "    This is an example of a callout with a title.\\n"\n   ]\n  },\n  {